# Quick Evaluation

In [60]:
import numpy as np 
import pandas as pd 
from pymoo.indicators.hv import HV

In [61]:
class Evaluator_Solutions(): 
    def __init__(self,name_algorithm, name_problem, operators, n_ejec, n_obj):
        self.name_algorithm = name_algorithm
        self.name_problem = name_problem 
        self.operators = operators 
        self.n_ejec = n_ejec 
        self.n_obj  = n_obj 
        self.ref_point  = np.array([1.1]*n_obj)
        
    def get_max_min(self): 
        #20 ejecuciones, 5 operadores, 100 ultimas generaciones 
        max_matrix     = np.empty(shape=(self.n_ejec*len(self.operators), self.n_obj))
        min_matrix     = np.empty(shape=(self.n_ejec*len(self.operators), self.n_obj))
        for i,name_oprt in enumerate(self.operators): 
            Mega_F = np.load('Solutions/'+self.name_algorithm+'/'+self.name_problem+'_'+name_oprt+'_F.npy')
            for j in range(self.n_ejec): 
                F = Mega_F[j, -1]
                max_matrix[i*self.n_ejec+j,:] = np.max(F, axis=0)
                min_matrix[i*self.n_ejec+j,:] = np.min(F, axis=0)
        general_max = np.max(max_matrix, axis=0)
        general_min = np.min(min_matrix, axis=0)
        return general_max, general_min
    
    def minmaxScaler(self,X, general_max, general_min, new_max, new_min): 
        X_std = (X -general_min)/(general_max-general_min)
        X_scaled = X_std*(new_max-new_min) + new_min
        return X_scaled
    
    def get_opt_HyperVolume(self):
        self.values_HV  = np.empty(shape=(len(self.operators), self.n_ejec))
        general_max, general_min = self.get_max_min()
        ind = HV(ref_point=self.ref_point)
        for i, name_oprt in enumerate(self.operators): 
            Mega_F = np.load('Solutions/'+self.name_algorithm+'/'+self.name_problem+'_'+name_oprt+'_F.npy')
            for j in range(self.n_ejec): 
                F_norm = self.minmaxScaler(Mega_F[j, -1], general_max, general_min, 1, 0)
                self.values_HV[i,j] = ind(F_norm)
        return 
    
    def get_report(self, save_file=False):
        S = pd.DataFrame()
        for i, name_oprt in enumerate(self.operators):
            result = {'Algoritmo': self.name_algorithm, 
                      'Problema' : self.name_problem, 
                      'Operador': name_oprt, 
                      'Ejecucion': range(self.n_ejec), 
                      'Generacion': 249, 
                      'HV_gen': self.values_HV[i]} 
            result = pd.DataFrame(result)
            S = pd.concat([S, result], ignore_index=True)   
        if save_file:
            S.to_csv('Hypervolumes/'+self.name_algorithm+'_'+self.name_problem+'.csv', index=False) 
        return S 

In [62]:
def get_final_reports(algorithms, problems, objectives, operators, n_ejec): 
    for name_algorithm in algorithms: 
        for name_problem,n_obj in zip(problems, objectives): 
            evals = Evaluator_Solutions(name_algorithm, name_problem, operators, n_ejec, n_obj)
            evals.get_opt_HyperVolume()
            final_report = evals.get_report(save_file=True)
    return 

In [64]:
def merge_dataframes(name_algorithm, problems): 
    S = pd.DataFrame()
    for name_problem in problems: 
        rr = pd.read_csv('Hypervolumes/'+name_algorithm+'_'+name_problem+'.csv') 
        S  = pd.concat([S, rr])
    return S 

In [69]:
def conteo_winners(name_algorithm, problems, operators):
    dict_winners = {oprt: [0]*len(operators) for oprt in operators} 
    for name_problem in problems: 
        rr = pd.read_csv('Hypervolumes/'+name_algorithm+'_'+name_problem+'.csv') 
        rr = rr.groupby('Operador', as_index=False).agg({'HV_gen': ['mean', 'std']})
        rr.columns = list(map(lambda x: '_'.join(x).rstrip('_'), rr.columns)) 
        temp_order  =rr.sort_values(by='HV_gen_mean', ascending=False)['Operador']
        for i, oprt in enumerate(temp_order): 
            dict_winners[oprt][i] +=1
        
    return pd.DataFrame(dict_winners).transpose()   

In [65]:
algorithms   = ['nsgaii']
wfg_problems = ['wfg1', 'wfg2', 'wfg3', 'wfg4', 'wfg5', 'wfg6', 'wfg7', 'wfg8', 'wfg9']
operators    = ['baseline', 'sam_energy', 'sam_uniform', 'mut_dirichlet', 'cross_spx']
objectives   = [3]*len(wfg_problems)
#get_final_reports(algorithms, wfg_problems,objectives, operators, 20)

In [67]:
SS = merge_dataframes('nsgaii', wfg_problems)
SS.groupby(['Problema', 'Operador']).agg({'HV_gen': ['mean','std']})

HV_gen          
                            mean       std
Problema Operador                         
wfg1     baseline       1.201469  0.004969
         cross_spx      1.164755  0.011467
         mut_dirichlet  1.203599  0.005064
         sam_energy     1.198214  0.006829
         sam_uniform    1.202642  0.004741
wfg2     baseline       0.270301  0.018491
         cross_spx      0.240860  0.018044
         mut_dirichlet  0.661862  0.036978
         sam_energy     0.674562  0.026150
         sam_uniform    0.278789  0.007039
wfg3     baseline       0.527692  0.011136
         cross_spx      0.498000  0.010165
         mut_dirichlet  0.532422  0.013973
         sam_energy     0.529704  0.009566
         sam_uniform    0.535728  0.005089
wfg4     baseline       0.562178  0.010355
         cross_spx      0.456893  0.018311
         mut_dirichlet  0.571321  0.010106
         sam_energy     0.569962  0.009121
         sam_uniform    0.563372  0.009464
wfg5     baseline       0.453780  0.003687
         cross_spx      0.406780  0.008953
         mut_dirichlet  0.454902  0.005784
         sam_energy     0.453169  0.005089
         sam_uniform    0.451740  0.004524
wfg6     baseline       0.540296  0.007645
         cross_spx      0.406331  0.070428
         mut_dirichlet  0.525612  0.013891
         sam_energy     0.539292  0.009007
         sam_uniform    0.536539  0.007901
wfg7     baseline       0.655887  0.007939
         cross_spx      0.583259  0.027943
         mut_dirichlet  0.671038  0.005821
         sam_energy     0.655117  0.008433
         sam_uniform    0.656450  0.008299
wfg8     baseline       1.101806  0.016327
         cross_spx      0.892103  0.021972
         mut_dirichlet  0.982622  0.198166
         sam_energy     1.116164  0.016185
         sam_uniform    1.113333  0.018218
wfg9     baseline       0.576056  0.020597
         cross_spx      0.486786  0.020407
         mut_dirichlet  0.583150  0.033567
         sam_energy     0.693260  0.012597
         sam_uniform    0.553686  0.024275

In [100]:
SS = conteo_winners('nsgaii', wfg_problems, operators)
n = len(operators)
SS['Borda'] = [ row.to_numpy()@np.array(range(n, 0,-1)) for idx, row in SS.iterrows() ]
SS

,0,1,2,3,4,Borda
baseline,1,1,4,3,0,27
sam_energy,3,2,2,2,0,33
sam_uniform,1,3,3,2,0,30
mut_dirichlet,4,3,0,2,0,36
cross_spx,0,0,0,0,9,9


In [95]:
for idx, row in SS.iterrows(): 
    print(idx, row.to_numpy())

baseline [1 1 4 3 0]
sam_energy [3 2 2 2 0]
sam_uniform [1 3 3 2 0]
mut_dirichlet [4 3 0 2 0]
cross_spx [0 0 0 0 9]


In [84]:
print('Ganador por voto mayoritario:', SS.idxmax()[0], 'resultado:', SS.max()[0])

Ganador por voto mayoritario: mut_dirichlet resultado: 4
